In [1]:
# !pip install mecab-python3

# !pip install unidic-lite
# !pip install japanize-matplotlib

In [1]:
import MeCab
import unidic
import sys
import re
import unicodedata
import matplotlib.pyplot as plt
import tqdm
import japanize_matplotlib
import os
import collections
import glob
import pandas as pd

In [3]:
# Check if MeCab works
# t = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')

t = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')

sentence = 'すもももももももものうち'
print(t.parse(sentence))

すもも	名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
うち	名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ
EOS



### Here is a translation 
- すもも	名詞 (= noun),一般 (=general),*,*,*,*,すもも,スモモ,スモモ
- も	助詞 (=particle) ,係助詞 (=binding particle),*,*,*,*,も,モ,モ
- もも	名詞 (= noun),一般 (=general),*,*,*,*,もも,モモ,モモ
- も	助詞 (=particle) ,係助詞 (=binding particle),*,*,*,*,も,モ,モ
- もも	名詞 (= noun),一般 (=general),*,*,*,*,もも,モモ,モモ
- の	助詞 (=particle) ,連体化(=attributive),*,*,*,*,の,ノ,ノ
- うち	名詞(=noun),非自立(=non-indipendent),副詞可能(adverb)=,*,*,*,うち,ウチ,ウチ

In [4]:
#create the list of meaningless words such as "that" "year" "what" etc... 
with open("stopwords.txt", encoding='UTF-8') as f:
    stopwords = f.read().split("\n")
print(stopwords)

['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', '', 'ハイ', '', '', '上', '中', '下', '字', '', '', '年', '月', '日', '時', '分', '秒', '週', '火', '水', '木', '金', '土', '国', '都', '道', '府', '県', '市', '区', '町', '村', '', '', '各', '第', '方', '何', '的', '度', '文', '者', '体', '人', '他', '今', '部', '課', '係', '外', '類', '達', '気', '室', '口', '誰', '用', '界', '会', '首', '別', '話', '私', '屋', '店', '家', '場', '等', '見', '際', '観', 

In [5]:
#read all the file in "F:\Wikipedia\wikiextractor-master\wikiextractor\f_scientist"




In [15]:

def count_word(text, pos_tags, stopwords):
    
    words_list = []
    tagger = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')
    
    node = tagger.parseToNode(text)

    while node:
        features = node.feature.split(",")
        base = features[6]
        pos_tag = features[0]
        
        # remove "stopwords"
        if pos_tag in pos_tags and base not in stopwords:
            words_list.append((base, pos_tag))
            

        node = node.next
    count = collections.Counter(words_list)

    return count

# chart
# def draw_barh_chart(y, width):
#     fig, ax = plt.subplots(figsize=(8, 4.5))
#     ax.barh(y, width)

# #     fig.savefig("word_count.png")


if __name__ == "__main__":
    # read files
    #change HERE to try other files
    with open('F:\Wikipedia\wikiextractor-master\wikiextractor\people\people鈴木みのり (カーリング選手)', 'r', encoding='UTF-8') as files:
        text = files.read()
    
    # read stopwords
    with open("stopwords.txt",encoding='UTF-8') as f:
        stopwords = f.read().split("\n")

    pos_tags = ["名詞", "動詞", "形容詞"] 
    #  ["名詞", "動詞", "形容詞"]=  ["noun", "verb", "adj"]
    count = count_word(text, pos_tags, stopwords)

top100_words = count.most_common(100)
top100_words
    
top100 = pd.DataFrame(top100_words, columns = ['word_tuple', "count"]) 
top100
   
    
    
# Draw a chart
#top_words = count.most_common(10)
#words = list(map(lambda x: x[0], top_words))[::-1]
#counts = list(map(lambda x: x[1], top_words))[::-1]
#draw_barh_chart(words, counts)

,word_tuple,count
0,"(選手権, 名詞)",15
1,"(カーリング, 名詞)",13
2,"(ジュニア, 名詞)",11
3,"(チーム, 名詞)",11
4,"(世界, 名詞)",11
...,...,...
95,"(冬季, 名詞)",1
96,"(いる, 動詞)",1
97,"(女子, 名詞)",1
98,"(富士急, 名詞)",1


In [16]:
top100.head()

,word_tuple,count
0,"(選手権, 名詞)",15
1,"(カーリング, 名詞)",13
2,"(ジュニア, 名詞)",11
3,"(チーム, 名詞)",11
4,"(世界, 名詞)",11


In [17]:
top100 = top100.assign(
    word = lambda x: x["word_tuple"].apply(lambda wt: wt[0]),
    pos = lambda x: x["word_tuple"].apply(lambda wt: wt[1])
)

In [18]:
top100.head()

,word_tuple,count,word,pos
0,"(選手権, 名詞)",15,選手権,名詞
1,"(カーリング, 名詞)",13,カーリング,名詞
2,"(ジュニア, 名詞)",11,ジュニア,名詞
3,"(チーム, 名詞)",11,チーム,名詞
4,"(世界, 名詞)",11,世界,名詞


In [19]:
file_list = glob.glob("F:/Wikipedia/wikiextractor-master/wikiextractor/f_scientist/*") 

In [20]:
file_list[:5]

['F:/Wikipedia/wikiextractor-master/wikiextractor/f_scientist\\エイダ・ラブレス',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/f_scientist\\アイン・ランド',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/f_scientist\\ウィリアム・ハーシェル',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/f_scientist\\ヒルデガルト・フォン・ビンゲン',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/f_scientist\\アンゲラ・メルケル']

In [21]:
# read stopwords
with open("stopwords.txt",encoding='UTF-8') as f:
    stopwords = f.read().split("\n")

pos_tags = ["名詞", "動詞", "形容詞"] 
#  ["名詞", "動詞", "形容詞"]=  ["noun", "verb", "adj"]

In [22]:
dfs = []

In [24]:
for f in file_list:
    with open(f, 'r', encoding='UTF-8') as file:
        text = file.read()
    
    count = count_word(text, pos_tags, stopwords)
    
    top100_words = count.most_common(100)
    
    top100 = (
        pd
        .DataFrame(top100_words, columns = ['word_tuple', "count"])
        .assign(
            word = lambda x: x["word_tuple"].apply(lambda wt: wt[0]),
            pos = lambda x: x["word_tuple"].apply(lambda wt: wt[1])
        )
    )
    #print(top100.head())
    f_name = f.split("\\")[-1]
    full_csv_file = "F:/Wikipedia/wikiextractor-master/wikiextractor/f_scientist/csvs/" + f_name + "_words.csv"
    #print(f_name)
    top100.to_csv(full_csv_file, index=None)
    dfs.append(top100)

PermissionError: [Errno 13] Permission denied: 'F:/Wikipedia/wikiextractor-master/wikiextractor/f_scientist\\csvs'

In [25]:
overall = (
    pd
    .concat(dfs)
    .groupby(["word", "pos"])["count"]
    .sum()
    .reset_index()
    .sort_values("count", ascending=False)
)

In [34]:
overall[overall.pos == "名詞"].head(60)

,word,pos,count
10238,研究,名詞,8424
6195,学,名詞,5738
5906,大学,名詞,5730
7848,教授,名詞,4410
8033,日本,名詞,4372
7109,彼女,名詞,3866
6098,女性,名詞,3338
10404,科学,名詞,2626
4963,博士,名詞,2560
6230,学者,名詞,2558


In [35]:
overall[overall.pos == "動詞"].head(60)

,word,pos,count
85,いる,動詞,12138
859,れる,動詞,11420
47,ある,動詞,3320
845,られる,動詞,2286
11468,行う,動詞,1638
4763,務める,動詞,1332
5109,受ける,動詞,1330
518,できる,動詞,1090
9739,生まれる,動詞,894
6197,学ぶ,動詞,784
